## Hardware Corrections

Sitting somewhere between I/O and preprocessing, the methods in the notebook are intended to correct flaws in the data caused by hardware issues.

### Nonpositive or NaN values in amplitude

Sometimes, hardware errors can lead to nonpositive amplitude values or to missing data (NaN values). We can simulate this by loading an example dataset and altering the data.

In [ ]:
import matplotlib.pyplot as p
import numpy as np
import xarray as xr

import cedalion
import cedalion.dataclasses as cdc
import cedalion.datasets
import cedalion.sigproc.quality as quality
import pint_xarray

# Load the dataset
rec = cedalion.datasets.get_fingertappingDOT()

display(rec["amp"])

# Set values to simulate hardware issues
unit = rec["amp"].pint.units
chwl = dict(channel="S1D1", wavelength="850")
rec["amp"].sel(**chwl).isel(time=slice(8,10))[:] = np.nan * unit
rec["amp"].sel(**chwl).isel(time=slice(19,20))[:] = 0 * unit

# Plot first 30 sec
f, ax = p.subplots(1, 1, figsize=(12, 4))
ax.plot(rec["amp"].time.isel(time=slice(0,30)), rec["amp"].sel(**chwl).isel(time=slice(0,30)))
ax.set_title("Amplitude")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Amplitude (a.u.)")
p.show()

The amp_preproc function replaces nonpositive amp values and optionally fills NaNs using the xarray interpolate_na method. It replaces nonpositive values with a small magnitude and runs a rolling median filter over the data. This method works best for isolated nonpositive values.

In [ ]:
# Correct hardware issues
rec["amp_proc"] = quality.amp_preproc(rec["amp"], median_len=3, interp_nan=True)

# Plot first 30 sec
f, ax = p.subplots(1, 1, figsize=(12, 4))
ax.plot(rec["amp_proc"].time.isel(time=slice(0,30)), rec["amp_proc"].sel(**chwl).isel(time=slice(0,30)))
ax.set_title("Amplitude")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Amplitude (a.u.)")
p.show()